In [1]:
# 导入需要的包
import torch
from torch import nn, optim
import torchvision
import torch.nn.functional as F

In [2]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from torch.autograd import Variable

##### 这个脚本是在colab上面跑的，所以需要把数据挂载到云盘上

In [4]:
#将数据挂载的google云盘上面
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

##### 将PIL Image或者numpy.ndarray转化为torch.FloatTensor，shape为(C,H,W)，并且做归一化处理

##### 一开始的代码是直接使用transforms.ToTensor(),没有设计到对原始图像的像素填充然后裁剪。当时跑出来的成功率大概在80%，可以看一下上传的图片文件epoch_100，这张是当时的截图。后来在网上查到可以对图片做填充和随机裁剪来提高测试的精确度，所以做了以下的处理

In [7]:
transform_train = transforms.Compose([
        # 对原始32*32图像四周各填充4个0像素（40*40），然后随机裁剪成32*32
        transforms.RandomCrop(32, padding=4),

        # 按0.5的概率水平翻转图片
        transforms.RandomHorizontalFlip(),

        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

##### 加载CIFAR-10数据集，因为提前下载好并且上传到了google drive，所以download设置成了False，如果没有下载数据集，可以设为True

In [8]:
# CIFAR-10 Dataset
train_dataset = datasets.CIFAR10(root='./',
                              train=True,
                              transform=transform_train,
                              download=False)

In [9]:
test_dataset = datasets.CIFAR10(root='./',
                             train=False,
                             transform=transform_test)

##### 定义Resnet网络层

In [10]:
# ResNet Module
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [11]:
# 3x3 Convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)

##### 残差块
##### in_channels, out_channels：残差块的输入、输出通道数
##### 对第一层，in out channel都是64，其他层则不同
##### 对每一层，如果in out channel不同， stride是1，其他层则为2

In [12]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

##### 使用DataLoader进行分批

In [13]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=True)

##### 创建一个resnet网络的对象

In [14]:
resnet = ResNet(ResidualBlock, [2, 2, 2, 2])

###### 损失函数:这里用交叉熵
##### 优化器 这里用SGD

In [15]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)

##### 创建device使得后续的代码能放到这个device上面执行，如果GPU可用的话会使用GPU。

In [16]:
#device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

##### 使用训练集对模型进行训练，最早试过10次迭代，准确率为78%，一般在150次迭代左右会非常接近在目前这些参数设置下模型的最高准确率

In [17]:
# Training
for epoch in range(150):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.to(device))
        labels = Variable(labels.to(device))

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        # 清空上一轮梯度
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('epoch{} loss:{:.4f}'.format(epoch+1, loss.item()))
    
    # Decaying Learning Rate
    if (epoch + 1) % 20 == 0:
        lr /= 3
        optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)

epoch1 loss:1.0124
epoch2 loss:0.8544
epoch3 loss:0.8355
epoch4 loss:0.5197
epoch5 loss:0.8224
epoch6 loss:0.6326
epoch7 loss:0.6122
epoch8 loss:0.5970
epoch9 loss:0.4054
epoch10 loss:0.4780
epoch11 loss:0.4204
epoch12 loss:0.3513
epoch13 loss:0.4626
epoch14 loss:0.3662
epoch15 loss:0.4924
epoch16 loss:0.5615
epoch17 loss:0.4429
epoch18 loss:0.3515
epoch19 loss:0.4305
epoch20 loss:0.2862
epoch21 loss:0.1654
epoch22 loss:0.2306
epoch23 loss:0.3554
epoch24 loss:0.4070
epoch25 loss:0.2892
epoch26 loss:0.2423
epoch27 loss:0.2454
epoch28 loss:0.2665
epoch29 loss:0.1581
epoch30 loss:0.0998
epoch31 loss:0.2675
epoch32 loss:0.1852
epoch33 loss:0.2174
epoch34 loss:0.2671
epoch35 loss:0.2217
epoch36 loss:0.1515
epoch37 loss:0.1880
epoch38 loss:0.1888
epoch39 loss:0.2499
epoch40 loss:0.1178
epoch41 loss:0.2110
epoch42 loss:0.2294
epoch43 loss:0.2334
epoch44 loss:0.1310
epoch45 loss:0.1339
epoch46 loss:0.2316
epoch47 loss:0.0925
epoch48 loss:0.0965
epoch49 loss:0.1088
epoch50 loss:0.2162
epoch51 l

##### 保存训练模型，实际上没有成功，在google drive下没有找到相应的模型文件，其他本地跑的代码可以生成pt文件。

In [18]:
torch.save(resnet, 'cifar_resnet.pt')
resnet2 = torch.load('cifar_resnet.pt')

In [19]:
# 测试
resnet.eval()
correct = 0
total = 0

In [20]:
for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    # 前向传播
    out = resnet(images)
    _, predicted = torch.max(out.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [21]:
#输出识别准确率
print('10000测试图像 准确率:{:.4f}%'.format(100 * correct / total)) 


10000测试图像 准确率:88.6400%
